### Morten model calculations

The following uses the Morten model to calcultate the total default probability of a company with given asset value and several debts which are to be paid at different times.
It calculates the total default probability as a product of individual default probabilities, 

In [ ]:
import numpy as np
from scipy.stats import norm

def merton_model(V0, D, T, r, sigma):
    # Calculates d1, d2, default probability, and equity value for a single debt.
    d1 = (np.log(V0 / D) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    default_prob = norm.cdf(-d2)
    equity_value = V0 * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2)
    return d1, d2, default_prob, equity_value

def sequential_default_probability(V0, debts, maturities, r, sigma):
    # Calculates the total default probability considering sequential debt repayment and risk-free growth between maturities.

    total_default_prob = 0
    remaining_assets = V0
    prev_maturity = 0  # To track the last maturity time

    print("Debt\tMaturity\tD1\tD2\tDefault Prob\tEquity Value\tRemaining Assets")

    # Process debts in the order of maturity
    for D, T in sorted(zip(debts, maturities), key=lambda x: x[1]):
        # Grow the remaining assets risk-free between maturities
        if T > prev_maturity:
            growth_period = T - prev_maturity
            remaining_assets *= np.exp(r * growth_period)
            prev_maturity = T

        # If assets are less than debt, the company defaults
        if remaining_assets < D:
            total_default_prob = 1.0
            print(f"{D}\t{T}\t\tN/A\t\tN/A\t\t100.00%\t\tN/A\t\t{remaining_assets:.2f}")
            break

        # Calculate the Merton model values
        d1, d2, default_prob, equity_value = merton_model(remaining_assets, D, T, r, sigma)
        
        # Print current debt data
        print(f"{D}\t{T}\t\t{d1:.4f}\t{d2:.4f}\t{default_prob:.3%}\t\t{equity_value:.2f}\t\t{remaining_assets:.2f}")

        # Accumulate default probability
        total_default_prob += default_prob * (1 - total_default_prob)
        
        # Repay the debt and reduce asset value
        remaining_assets -= D

    print(f"\nTotal Default Probability (Adjusted with Growth): {total_default_prob:.4%}")

# Example Parameters
V0 = 500  # Initial asset value
debts = [80, 90, 100, 110, 120]  # Debt face values
maturities = [1, 2, 3, 4, 5]  # Maturity times in years
r = 0.05  # Risk-free rate
sigma = 0.3  # Volatility

sequential_default_probability(V0, debts, maturities, r, sigma)


Debt	Maturity	D1	D2	Default Prob	Equity Value	Remaining Assets
80	1		6.5919	6.2919	0.000%		449.54		525.64
90	2		4.3362	3.9119	0.005%		387.05		468.48
100	3		3.2062	2.6866	0.361%		311.86		397.89
110	4		2.3771	1.7771	3.778%		223.77		313.16
120	5		1.5675	0.8967	18.494%		124.91		213.58

Total Default Probability (Adjusted with Growth): 21.8602%
